In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
import cv2
from PIL import Image
import imageio
from IPython.display import HTML

# Table of Contents




* [Exploration](#section-one)


* [Case](#case_section)
    - [Case Selection](#case_selection)
    - [Stacked Gif](#stacked_gif)
    

<a id="section-one"></a>
# Exploration

In [ ]:
IMG_PATH = "../input/uw-madison-gi-tract-image-segmentation/train/"# folder are name as case
train = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/train.csv')
test = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/sample_submission.csv')


In [ ]:
def create_pie(df, target_variable, figsize=(10, 10)):
    print(df[target_variable].value_counts())
    fig, ax = plt.subplots(figsize=figsize)
    ax.pie(df[target_variable].value_counts().values, labels=df[target_variable].value_counts().index, autopct='%1.2f%%', textprops={'fontsize': 10})
    ax.axis('equal')
    plt.title(target_variable)
    plt.show()

In [ ]:

# DATASET GLOBAL TRANSFORMATION

item_box = []
for item in train['id']:
    items = str(item)
    new_item = items.replace('_d', ' d').replace('_s', ' s')
    clean = new_item.split()
    item_box.append(clean)
    

cases = []
days = []
slices = []
for item in item_box:
    cases.append(item[0])
    days.append(item[1])
    slices.append(item[2])

day_numbers = []
for item in days:
    items = str(item)
    new_item = items.replace('day', '')
    day_numbers.append(int(new_item))
    
slice_numbers = []
for item in slices:
    items = str(item)
    new_item = items.replace('slice_', '')
    slice_numbers.append(int(new_item))
    
case_numbers = []
for item in cases:
    items = str(item)
    new_item = items.replace('case', '')
    case_numbers.append(int(new_item))

# I need to encode class
stomach_class      = [1, 0, 0]
large_bowel_class  = [0, 1, 0]
small_bowel_class = [0, 0, 1]

stomach_class_n      = 0
large_bowel_class_n  = 1
small_bowel_class_n  = 2

encoded = []
numerical_encoded = []

for item in train['class']:
    if str(item) == 'stomach':
        encoded.append(stomach_class)
        numerical_encoded.append(stomach_class_n)
        
    elif str(item) == 'large_bowel':
        encoded.append(large_bowel_class)
        numerical_encoded.append(large_bowel_class_n)
        
    elif str(item) == 'small_bowel':
        encoded.append(small_bowel_class)
        numerical_encoded.append(small_bowel_class_n)
        
    else:
        print('somethings went wrong when encoding class')

        
        
        
# list to dataframes        
class_encode = pd.Series(encoded)
class_numerical_encode = pd.Series(numerical_encoded)
cases = pd.Series(cases)
case_int = pd.Series(case_numbers)
days = pd.Series(days)
day_number = pd.Series(day_numbers)
slicing = pd.Series(slices)
slice_number = pd.Series(slice_numbers)

train['classencoded'] = class_encode
train['classnumericalencoded'] = class_numerical_encode
train['case'] = cases
train['caseintegrer'] = case_int
train['day'] = days
train['daynumber'] = day_number
train['slice'] = slicing
train['slicenumber'] = slice_number

print(train.shape)
train

In [ ]:
# Put Image in Dataset


cases = train['case'].tolist()
days = train['day'].tolist()
slices = train['slice'].tolist()
scans = 'scans'
cases = cases
paths = []
files_name = []

for i, item in enumerate(cases):
    day = days[i]
    folder_path = f'{IMG_PATH}{item}/{item}_{day}/{scans}/'
    
    arr = os.listdir(folder_path)
    for file in arr:
        if str(file[:int(len(str(slices[i])))]) == str(slices[i]):
            img_path = f'{IMG_PATH}{item}/{item}_{day}/{scans}/{file}'
            files_name.append(file)
            paths.append(img_path)

files = pd.Series(files_name)
paths = pd.Series(paths)

train['filename'] = files
train['imagepath'] = paths


# here all work but need some optimization

del cases
del days
del slices
del scans
del paths
del files_name
del files

train.head()

<a id="case_section"></a>
# Case


In [ ]:
# I want to link segmentation organs by day then by slice and put mask on the same image with they're own segmentation and color
# but first I need to check where are missing value and how many value is missing by case and also where in class



plt.figure(figsize=(20, 20))
case = train.copy()
print(case['case'].value_counts(normalize=True).sort_values(ascending=False))
#case['case'].value_counts(normalize=True).plot.pie();



In [ ]:
# Bigger case
case36 = case.loc[case['case'] == 'case36']#.set_index('daynumber')

# Smaller Case
case116 = case.loc[case['case'] == 'case116'].set_index('daynumber')

<a id="case_selection"></a>
# Case Selection

In [ ]:
### Bigger Case
case36
# % of missing value
#print((case36.isna().sum() / case36.shape[0]).sort_values(ascending=False))

case36_class = case36['class'].to_list()
case36_nan_class = []
case36_nan_indexes = []
for i, item in enumerate(case36['segmentation']):
    if str(item) == 'nan':
        #print(case36_class[i])
        case36_nan_class.append(case36_class[i])
        

#nan_class = pd.Series(case36_nan_class)

#print(nan_class.value_counts(normalize=True).sort_values(ascending=False), '\n\n')
#plt.figure(figsize=(8, 8))
#plt.title(f'Case36:  % Missing Value')
#nan_class.value_counts(normalize=True).sort_values(ascending=False).plot.pie();

### 

In [ ]:
pie = case36.drop(['id'], axis=1)
create_pie(pie, 'class')

In [ ]:
case36.shape

In [ ]:
case36_nanless = case36.copy()
case36_nanless.dropna(axis=0,inplace=True)
case36_nanless.shape

In [ ]:

plt.figure(figsize=(25, 6))
plt.scatter(case36_nanless['slicenumber'], case36_nanless['daynumber']);

#slicenumber
#daynumber

In [ ]:

case36_nanless_day6 = case36_nanless.copy()
case36_nanless_day6 = case36_nanless_day6.loc[case36_nanless_day6['daynumber'] == 6]

case36_nanless_day6

plt.figure(figsize=(30, 6))
plt.scatter(case36_nanless_day6['slicenumber'], case36_nanless_day6['classnumericalencoded']);


In [ ]:
# Here I have 2 Idea 
#
# First:
# category 0 can be on the first layer then disapear for the orther but still here,  meaning of this Idea all Data at this rate is correct and all Label are here

# Second:
# category 0 can be on the first layer and are not drawing and I keep data between 55 -> 61,    I don't like this Idea

In [ ]:
img = pltimg.imread('../input/uw-madison-gi-tract-image-segmentation/train/case36/case36_day6/scans/slice_0056_266_266_1.50_1.50.png')

# ref.: https://www.kaggle.com/stainsby/fast-tested-rle
def mask2rle(img):
    """
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formatted
    """
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def rle2mask(mask_rle: str, label=1, shape=img.shape):
    """
    mask_rle: run-length as string formatted (start length)
    shape: (height,width) of array to return
    Returns numpy array, 1 - mask, 0 - background

    """
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = label
    return img.reshape(shape)  # Needed to align to RLE direction

In [ ]:
case36_nanless_day6_slice56 = case36_nanless_day6.copy()
case36_nanless_day6_slice56 = case36_nanless_day6_slice56.loc[case36_nanless_day6_slice56['slicenumber'] == 56]

case36_nanless_day6_slice56





In [ ]:

case36_nanless_day6_slice56_segmentation = case36_nanless_day6_slice56['segmentation'].tolist()
case36_nanless_day6_slice56_imgpath = case36_nanless_day6_slice56['imagepath'].tolist()
columns = 3

In [ ]:
plt.figure(figsize=(25, 25))

for i, mask in enumerate(case36_nanless_day6_slice56_segmentation):
    
    plt.subplot(int(len(case36_nanless_day6_slice56_segmentation) / columns + 1), columns, i + 1)
    #rle2mask(mask_rle: str, label=1, shape=img.shape):
    img = cv2.imread(f'{case36_nanless_day6_slice56_imgpath[i]}')
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    # Now create a mask and create its inverse mask also
    mask = rle2mask(str(mask), label=1, shape=gray.shape)
    #mask_inv = cv2.bitwise_not(mask)

    
    
    # Now black-out the area
    #img1_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)
    img1_bg = gray
    #plt.figure(figsize=(28, 8))
    #plt.imshow(img1_bg, cmap='gray')
    
    
    # Take only region 
    img2_fg = cv2.bitwise_and(gray,gray,mask = mask)
    #plt.figure(figsize=(28, 8))
    #plt.imshow(img2_fg, cmap='gray')
    
    
    #modify the main image
    dst = cv2.add(img1_bg,img2_fg)

    #plt.figure(figsize=(28, 8))
    #plt.imshow(dst,cmap = "mako")
    result = np.hstack((img1_bg,img2_fg))
    plt.axis("off")
    plt.imshow(result, cmap='mako')

In [ ]:

case36_nanless_day6_rle = case36_nanless_day6['segmentation'].tolist()
case36_nanless_day6_imgpath = case36_nanless_day6['imagepath'].tolist()
case36_nanless_day6_class = case36_nanless_day6['class'].tolist()

slices = case36_nanless_day6['slicenumber']

last_slice = [0]

frames = []
partial_frames = []
columns = 3
for i, item in enumerate(slices):
    #print(item)
    if int(item) == int(last_slice[-1]) or int(item) == 0:
        partial_frames.append(f'{case36_nanless_day6_class[i]}')
        
        
    else:
        if i > 0:
            partial_frames.append(f'{case36_nanless_day6_class[i]:-<30}slice: {item}')
            frames.append(partial_frames)
            partial_frames = []
            
    last_slice.append(item)
    
frames # data distribution for day by slice for case 36
#print(f'{col :-<50} {train[col].unique()}')

In [ ]:
#plt.figure(figsize=(25, 400))
#columns = 3
case36_nanless_day6_rle = case36_nanless_day6['segmentation'].tolist()
case36_nanless_day6_imgpath = case36_nanless_day6['imagepath'].tolist()
case36_nanless_day6_class = case36_nanless_day6['class'].tolist()
slices = case36_nanless_day6['slicenumber']

last_slice = [0]
frames = []
partial_frames = []

for i, item in enumerate(slices):
    #plt.subplot(int(len(case36_nanless_day6_rle) / columns + 1), columns, i + 1)
    
    color_img = cv2.imread(f'{case36_nanless_day6_imgpath[i]}')
    gray = cv2.cvtColor(color_img,cv2.COLOR_BGR2GRAY)
    
    # Now create a mask 
    mask = rle2mask(str(case36_nanless_day6_rle[i]), label=1, shape=gray.shape)
    
    if int(item) == int(last_slice[-1]) or int(item) == 0:
        foreground = cv2.bitwise_and(gray,gray,mask = mask)
        partial_frames.append(foreground)
        
        
    else:
        if i > 0:
            foreground = cv2.bitwise_and(gray,gray,mask = mask)
            partial_frames.append(foreground)

            frames.append(partial_frames)
            partial_frames = []
            
    last_slice.append(item)
    


In [ ]:
# from frames create foreground frame


#plt.figure(figsize=(25, 400))

foreground_frames = []
columns = 3
for i, item in enumerate(frames):
    #plt.subplot(int(len(frames) / columns + 1), columns, i + 1)
    #print(len(item))
    if len(item) == 1:
        #plt.imshow(item[0], cmap = 'mako')
        foreground_frames.append(item[0])
        
    elif len(item) == 2:
        foreground = item[0] + item[1]
        #plt.imshow(foreground, cmap = 'mako')
        foreground_frames.append(foreground)
        
    elif len(item) == 3:
        foreground = item[0] + item[1]
        foreground = foreground  + item[2]
        #plt.imshow(foreground, cmap = 'mako')
        foreground_frames.append(foreground)

In [ ]:
#train_rle = train['segmentation'].tolist()
#train_imgpath = train['imagepath'].tolist()
#train_class = train['class'].tolist()
#slices = train['slicenumber']
#slices = slices[:3000]


case36_nanless_day6_rle     = case36_nanless_day6['segmentation'].tolist()
case36_nanless_day6_imgpath = case36_nanless_day6['imagepath'].tolist()
case36_nanless_day6_class   = case36_nanless_day6['class'].tolist()
slices                      = case36_nanless_day6['slicenumber']


last_slice = [0]


slice_image = []
slice_numbers = []
frames = []
partial_frames = []

foreground_frames = []
partial_foreground_frames = []
#plt.figure(figsize=(25, 25))


r = [100,0,0]
g = [0,100,0]
b = [0,0,100]


#SIZE = 128
SIZE = 256

columns = 3
for i, item in enumerate(slices):
    # put color on mask
    if str(case36_nanless_day6_class[i]) == "stomach":
        color = r
    elif str(case36_nanless_day6_class[i]) == "large_bowel":
        color = g
    elif str(case36_nanless_day6_class[i]) == "small_bowel":
        color = b
    else:
        pass
    
    
    
    #plt.subplot(int(len(slices) / columns + 1), columns, i + 1)
    # load images
    color_img = cv2.imread(f'{case36_nanless_day6_imgpath[i]}', 1)
    gray = pltimg.imread(f'{case36_nanless_day6_imgpath[i]}')

    # Now create a mask and create its inverse mask also
    mask = rle2mask(str(case36_nanless_day6_rle[i]), label=1, shape=gray.shape)
    #foreground = cv2.bitwise_and(gray,gray,mask = mask)
    foreground = cv2.bitwise_and(color_img,color_img,mask = mask)
    
    # resize our image with pillow library
    p_mask = Image.fromarray(mask)
    p_gray = Image.fromarray(color_img)
    p_foreground = Image.fromarray(foreground)
    
    colored_mask = cv2.bitwise_and(color_img,color_img,mask = mask)
    colored_mask[mask >= 1] = color
    p_mask = Image.fromarray(colored_mask)
    
    new_size = (SIZE, SIZE)
    
    new_gray = color_img.copy()
    new_gray = p_gray.resize(new_size)
    
    new_mask = colored_mask.copy()
    new_mask = p_mask.resize(new_size)
    
    new_foreground = foreground.copy()
    new_foreground = p_foreground.resize(new_size)
    
    
    
    # convert them into a numpy array
    np_mask = np.array(new_mask)
    np_gray = np.array(new_gray)
    np_foreground = np.array(new_foreground)
    
    if int(item) == int(last_slice[-1]) or int(item) == 0:
        #partial_frames.append(mask)
        mask = np_mask.reshape(np_mask.shape[0], np_mask.shape[1], np_mask.shape[2])
        partial_frames.append(np.array(mask))
        
        foreground = np_foreground.reshape(np_foreground.shape[0], np_foreground.shape[1], np_foreground.shape[2])
        partial_foreground_frames.append(np.array(foreground))
        
        
    else:
        if i > 0:
            mask = np_mask.reshape(np_mask.shape[0], np_mask.shape[1], np_mask.shape[2])
            partial_frames.append(np.array(mask))
            frames.append(partial_frames)
            partial_frames = []
        
            foreground = np_foreground.reshape(np_foreground.shape[0], np_foreground.shape[1], np_foreground.shape[2])
            foreground_frames.append(partial_foreground_frames)
            partial_foreground_frames = []
            
            gray = np_gray.reshape(np_gray.shape[0], np_gray.shape[1], np_gray.shape[2])
            slice_image.append(gray)
            
            slice_numbers.append(item)
            
    last_slice.append(item)
    

In [ ]:
train_image = slice_image.copy()
train_mask = frames.copy()
train_number = slice_numbers.copy()
train_foreground = foreground_frames.copy()

print(len(train_image))
print(len(train_mask))
print(len(train_number))
print(len(train_foreground))

In [ ]:
stacked_mask = []

#plt.figure(figsize=(25, 25))
for i, mask in enumerate(train_mask):# and i <= 20:
    #plt.subplot(int(len(train_image) / columns + 1), columns, i + 1)
    
    #print(len(mask))
    
    if len(mask) == 1:
        dst = train_mask[i][0]
        #plt.imshow(train_mask[i][0])
        stacked_mask.append(np.array(dst))
        
    elif len(mask) == 2:
        dst = cv2.add(train_mask[i][0],train_mask[i][1])
        #plt.imshow(dst)
        stacked_mask.append(np.array(dst))
        
    elif len(mask) == 3:
        dst = cv2.add(train_mask[i][0],train_mask[i][1], train_mask[i][2])
        #plt.imshow(dst)
        stacked_mask.append(np.array(dst))
        
stacked_foreground = []

#plt.figure(figsize=(25, 25))
for i, mask in enumerate(train_foreground):# and i <= 20:
    #plt.subplot(int(len(train_image) / columns + 1), columns, i + 1)
    
    #print(len(mask))
    
    if len(mask) == 1:
        dst = train_foreground[i][0]
        #plt.imshow(train_mask[i][0])
        stacked_foreground.append(np.array(dst))
        
    elif len(mask) == 2:
        dst = cv2.add(train_foreground[i][0],train_foreground[i][1])
        #plt.imshow(dst)
        stacked_foreground.append(np.array(dst))
        
    elif len(mask) == 3:
        dst = cv2.add(train_foreground[i][0],train_foreground[i][1], train_foreground[i][2])
        #plt.imshow(dst)
        stacked_foreground.append(np.array(dst))

In [ ]:

train = train_image.copy()
gif_frames = []

for i, image in enumerate(stacked_foreground):
    img = train_image[i]
    mask = stacked_mask[i]
    foreground = stacked_foreground[i]
    
    
    gray_mask = cv2.cvtColor(mask,cv2.COLOR_BGR2GRAY)
    gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray_foreground = cv2.cvtColor(foreground,cv2.COLOR_BGR2GRAY)
    #np.expand_dims(gray)
    gray_img = gray_img.reshape(gray_img.shape[0], gray_img.shape[1], 1)
    gray_mask= gray_mask.reshape(gray_mask.shape[0], gray_mask.shape[1], 1)
    gray_foreground= gray_foreground.reshape(gray_foreground.shape[0], gray_foreground.shape[1], 1)
    '''print(gray_img.shape)
    print(gray_mask.shape)
    print(gray_foreground.shape)'''
    
    frames = np.hstack((gray_foreground, gray_img, gray_mask))
    gif_frames.append(frames)
    '''plt.imshow(frames)
    plt.show()'''

In [ ]:
filenames = []
for i, item in enumerate(gif_frames):
    #print(len(item))
    plt.axis("off")
    plt.imshow(item)
    
    # create file name and append it to a list
    filename = f'{i}.png'
    filenames.append(filename)
    
    # save frame
    plt.savefig(filename)
    plt.close()
    
    
 

# build gif
with imageio.get_writer('stacked.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
        
# Remove files
for filename in set(filenames):
    os.remove(filename)

<a id="stacked_gif"></a>
# Stacked Gif

In [ ]:
# https://www.kaggle.com/code/subinium/how-to-use-matplotlib-animations-in-kaggle


HTML('<img src="./stacked.gif" />')

In [ ]:
# Smaller Case
case116
(case116.isna().sum() / case116.shape[0]).sort_values(ascending=False)



case116_class = case116['class'].to_list()
case116_nan_class = []
case116_nan_indexes = []
for i, item in enumerate(case116['segmentation']):
    if str(item) == 'nan':
        #print(case36_class[i])
        case116_nan_class.append(case116_class[i])
        

nan_class = pd.Series(case116_nan_class)

print(nan_class.value_counts(normalize=True).sort_values(ascending=False), '\n\n')
plt.figure(figsize=(8, 8))
plt.title(f'Case116:  % Missing Value')
nan_class.value_counts(normalize=True).sort_values(ascending=False).plot.pie();